#Mount google drive

In [1]:
try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    base_path = "/content/gdrive/My Drive"
except:
    base_path = "C:/Users/Schiller/Google Drive/"
%cd {base_path}

Mounted at /content/gdrive
/content/gdrive/My Drive


In [8]:
import os
trained_dir     = base_path + "/trained_models"
model_name      = "efficientdet-d0_2020_11_10-16"

input_saved_model_dir = os.path.join(trained_dir, model_name, "saved_model")

output_saved_model_dir      = base_path + "/trained_model_trt"
if not os.path.exists(output_saved_model_dir):
    os.mkdir(output_saved_model_dir)
output_saved_model_dir = os.path.join(output_saved_model_dir, model_name)

#Convert the model

In [18]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.python.compiler.tensorrt import trt_convert as trt

#input_saved_model_dir = './efficientdet_d1_coco17_tpu-32/saved_model/'
#output_saved_model_dir = './models/tensorRT/'
num_runs = 2

conversion_params = trt.DEFAULT_TRT_CONVERSION_PARAMS
conversion_params = conversion_params._replace(max_workspace_size_bytes=(1<<32))
conversion_params = conversion_params._replace(precision_mode="FP16")
# conversion_params = conversion_params._replace(maximum_cached_engiens=100)

converter = trt.TrtGraphConverterV2(input_saved_model_dir=input_saved_model_dir,
                                    conversion_params=conversion_params)
converter.convert()

def my_input_fn():
    for _ in range(num_runs):
        inp1 = keras.Input(shape=(640,640,3,))
        yield (inp1,)
        
converter.build(input_fn=my_input_fn)
converter.save(output_saved_model_dir)

INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)


_SymbolicException: ignored

In [ ]:


# convert (optimize) frozen model to TensorRT model
trt_graph = trt.create_inference_graph(
    input_graph_def=frozen_graph,# frozen model
    outputs=your_outputs,
    max_batch_size=2,# specify your max batch size
    max_workspace_size_bytes=2*(10**9),# specify the max workspace
    precision_mode="FP32") # precision, can be "FP32" (32 floating point precision) or "FP16"

#write the TensorRT model to be used later for inference
with gfile.FastGFile("./model/TensorRT_model.pb", 'wb') as f:
    f.write(trt_graph.SerializeToString())
print("TensorRT model is successfully stored!")